In [1]:
import pandas as pd
from dados import Dados
from carga_dados import CargaDados
from limpeza_dados import LimpezaDados
from preprocessamento import Preprocessamento

In [2]:
pd.set_option('max_colwidth', 180)

## Carga de Dados

In [3]:
dados = Dados()
seg_orig = dados.seg_orig
seg_limp = dados.seg_limp
seg_prep = dados.seg_prep

In [4]:
cg = CargaDados()
cg.executar(dados)

INFO:root:Carregando dados de segmentos...
INFO:root:206488 registros carregados.


## Limpeza de Dados

In [5]:
lp = LimpezaDados(dados)
lp.executar(dados)

INFO:root:Excluindo segmentos dos atos que não são ADE...
INFO:root:91448 segmentos de atos não ADE excluídos.
INFO:root:Restaram 115040 segmentos de atos ADE.
INFO:root:Removendo segmentos não representativos...
INFO:root:6827 segmentos não representativos excluídos.
INFO:root:Restaram 108213 segmentos representativos.
INFO:root:Removendo segmentos nulos...
INFO:root:69 segmentos nulos excluídos.
INFO:root:Restaram 108144 segmentos não nulos.
INFO:root:Removendo tags HTML...
INFO:root:Removendo caracteres de escape HTML...
INFO:root:Reclassificando segmentos não identificados...
INFO:root:1418 segmentos reclassificados como artigos.
INFO:root:Limpeza de dados concluída.


## Preprocessamento

In [6]:
pp = Preprocessamento()
pp.executar(dados)

INFO:root:Carregando modelo de Português para preprocessamento...
INFO:root:Executando pipeline de preprocessanmento...


In [7]:
dados.seg_prep.sample()

,id_ato,data_pub,tipo_ato,id_seg,tipo_seg,txt_seg,docs
177179,111699,2020-08-12,ADE,2172981,Ementa,Autoriza o fornecimento de selos de controle de bebidas para importação.,"(Autoriza, o, fornecimento, de, selos, de, controle, de, bebidas, para, importação, .)"
